In [ ]:
#Создаем config для подключения к нашей базе данных с помощью имеющихся данных о сервере:

In [32]:
from configparser import ConfigParser
 
def read_db_config(filename='config.ini', section='mysql'):
    
    parser = ConfigParser()
    parser.read(filename)
 
    db = {}
    if parser.has_section(section):
        items = parser.items(section)
        for item in items:
            db[item[0]] = item[1]
    else:
        raise Exception('{0} not found in the {1} file'.format(section, filename))
 
    return db

In [34]:
from python_mysql_dbconfig import read_db_config
read_db_config()

{'database': 'python_mysql',
 'host': 'localhost',
 'password': 'root',
 'user': 'root'}

In [ ]:
#Проверяем возможность подключения к нашей базе данных:

In [66]:
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config
 
def connect():
    db_config = read_db_config()
 
    try:
        print('Connecting to MySQL database...')
        conn = MySQLConnection(**db_config)
 
        if conn.is_connected():
            print('connection established.')
        else:
            print('connection failed.')
 
    except Error as error:
        print(error)
 
    finally:
        conn.close()
        print('Connection closed.')

if __name__ == '__main__':
    connect()

Connecting to MySQL database...
connection established.
Connection closed.


In [ ]:
#Модуль вызова данных из определенной таблицы нашей БД:

In [75]:
def iter_row(cursor, size=10):
    while True:
        rows = cursor.fetchmany(size)
        if not rows:
            break
        for row in rows:
            yield row
            
def query_with_fetchmany():
    try:
        dbconfig = read_db_config()
        conn = MySQLConnection(**dbconfig)
        cursor = conn.cursor()
 
        cursor.execute("SELECT * FROM books")
 
        for row in iter_row(cursor, 10):
            print(row)
 
    except Error as e:
        print(e)
 
    finally:
        cursor.close()
        conn.close()
        
if __name__ == '__main__':
    query_with_fetchmany()

(1, 'Bel and the Dragon ', '123828863494')
(2, 'Daughters of Men ', '1234404543724')
(3, 'The Giant on the Hill ', '1236400967773')
(4, 'Marsh Lights ', '1233673027750')
(5, 'Mr. Wodehouse and the Wild Girl ', '1232423190947')
(6, 'The Fairy Castle ', '1237654836443')
(7, 'The Girl Who Walked a Long Way ', '1230211946720')
(8, 'The Runaway ', '1238155430735')
(9, 'The Shrubbery ', '1237366725549')
(10, 'Tom Underground a play ', '1239633328787')
(11, 'Anemones of the British Coast ', '1233540471995')
(12, 'Ask to Embla poem-cycle ', '1237417184084')
(13, 'Cassandra verse drama ', '1235260611012')
(14, 'Chidiock Tichbourne ', '1230468662299')
(15, 'The City of Is ', '1233136349197')
(16, 'Cromwell verse drama ', '1239653041219')
(17, 'Debatable Land Between This World and the Next ', '1235927658929')
(18, 'The Fairy Melusina epic poem ', '1232341278470')
(19, 'The Garden of Proserpina ', '1234685512892')
(20, 'Gods Men and Heroes ', '1233369260356')
(21, 'The Great Collector ', '1237871

In [ ]:
#Модуль вставки одной строки в определенную таблицу нашей БД:

In [74]:
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config
 
def insert_book(title, isbn):
    query = "INSERT INTO books(title,isbn)""VALUES(%s,%s)"
    args = (title, isbn)
 
    try:
        db_config = read_db_config()
        conn = MySQLConnection(**db_config)
 
        cursor = conn.cursor()
        cursor.execute(query, args)
 
        if cursor.lastrowid:
            print('last insert id', cursor.lastrowid)
        else:
            print('last insert id not found')
 
        conn.commit()
    except Error as error:
        print(error)
 
    finally:
        cursor.close()
        conn.close()

def main():
    insert_book ("Harry Potter And The Philosopher's Stone",'9781579187036')

if __name__ == '__main__':
    main()

last insert id 93


In [ ]:
#Модуль вставки нескольких строк в определенную таблицу нашей БД:

In [76]:
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config
 
def insert_books(books):
    query = "INSERT INTO books(title,isbn)""VALUES(%s,%s)"
 
    try:
        db_config = read_db_config()
        conn = MySQLConnection(**db_config)
 
        cursor = conn.cursor()
        cursor.executemany(query, books)
 
        conn.commit()
    except Error as e:
        print('Error:', e)
 
    finally:
        cursor.close()
        conn.close()

def main():
    books = [('Harry Potter And The Goblet of Fire', '9780432358071'),
             ('Harry Potter And The Prisoner of Azkaban', '9780432740071'),
             ('Harry Potter And The Half-Blood Prince', '9747319783268')]
    insert_books(books)

if __name__ == '__main__':
    main()

In [ ]:
#Модуль обновления данных из определенной таблицы нашей БД:

In [72]:
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config
 
def update_book(book_id, title):

    db_config = read_db_config()
 
    query = """ UPDATE books
                SET title = %s
                WHERE id = %s """
 
    data = (title, book_id)
 
    try:
        conn = MySQLConnection(**db_config)
 
        cursor = conn.cursor()
        cursor.execute(query, data)
 
        conn.commit()
 
    except Error as e:
        print('Error:', e)
 
    finally:
        cursor.close()
        conn.close()
        
if __name__ == '__main__':
    update_book(37, 'The Giant on the Hill *** TEST ***')

In [ ]:
#Модуль удаления данных из определенной таблицы нашей БД:

In [73]:
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config
 
def delete_book(book_id):
    db_config = read_db_config()
 
    query = "DELETE FROM books WHERE id = %s"
 
    try:
        conn = MySQLConnection(**db_config)
 
        cursor = conn.cursor()
        cursor.execute(query, (book_id,))
 
        conn.commit()
 
    except Error as error:
        print(error)
 
    finally:
        cursor.close()
        conn.close()

if __name__ == '__main__':
    delete_book(45)